In [1]:
!nvidia-smi

Tue Mar 09 17:07:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106... WDDM  | 00000000:02:00.0  On |                  N/A |
|  0%   37C    P8     7W / 120W |    584MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

# Pre-Processing and Training the Model
This notebook goes through the steps in turning the cleaned data into a representation that can be used to train a pre-trained BERT model for sentiment analysis.

__Columns__
- `target` sentiment of tweet (0=positive, 1=negative)
- `tweet` cleaned tweet text

_Note_
_This notebook expects the zipped datasets to be in the ./resources sub-directory with the names train_twitter_data.zip, test_twitter_data, and validate_twitter_data (e.g., ./resources/train_twitter_data.zip). If the datasets do not exist open and run the `cleaning.ipynb` notebook to generate them._

In [2]:
import sys 
!{sys.executable} -m pip install tf-nightly --user --use-feature=2020-resolver
!{sys.executable} -m pip install tensorflow-text-nightly --user --use-feature=2020-resolver
!{sys.executable} -m pip install tf-models-nightly --user --use-feature=2020-resolver

In [3]:
import os
import shutil

import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization

tf.get_logger().setLevel('ERROR')

C:\Developer\lib\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:37: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20210309). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


## Read Files
Read the files into memory and convert them to TensorFlow datasets to begin pre-processing.

In [4]:
RESOURCES_PATH = './resources/{}'

def twitter_data_to_dataset(filename):
    pd_df = pd.read_csv(RESOURCES_PATH.format(filename), encoding = 'ISO-8859-1', compression='zip')
    tweets =  tf.constant(pd_df['tweet'].to_numpy(dtype='U'), dtype=tf.string)
    targets = tf.constant(pd_df['target'].to_numpy(dtype='i4'), dtype=tf.int32)
    raw_ds = tf.data.Dataset.from_tensor_slices((tweets, targets))
    return raw_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = twitter_data_to_dataset('train_twitter_data.zip')
test_ds = twitter_data_to_dataset('test_twitter_data.zip')
validate_ds = twitter_data_to_dataset('validate_twitter_data.zip')

In [5]:
for feat, targ in train_ds.take(5):
  print ('Features: {}'.format(feat))
  print ('Target: {}'.format(targ))

Features: b'how long you been on here is not too shabby youll get more'
Target: 1
Features: b'i have drunk hickups i had a pretty laid back great night but i still can not help but be mad about the situation i put myself in'
Target: 1
Features: b'yeahshe passed last week so sad'
Target: 1
Features: b'although the guy playing the dmeo is playing it alot better'
Target: 1
Features: b'no I am even more frustrated'
Target: 1


## Load BERT Resources

In [6]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


### Pre-Processor

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [8]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


### Model

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [10]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76262873  0.9928099  -0.1861185   0.3667384   0.15233701  0.6550448
  0.9681154  -0.9486271   0.00216167 -0.9877732   0.06842678 -0.9763059 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946328  0.3432129   0.3323155  ...  0.21300781  0.71020806
  -0.05771036]
 [-0.28742033  0.31981045 -0.2301859  ...  0.5845505  -0.21329637
   0.72692126]
 [-0.6615699   0.6887682  -0.87433034 ...  0.10877267 -0.26173192
   0.47855562]
 ...
 [-0.22561109 -0.28925675 -0.07064371 ...  0.47566092  0.8327719
   0.40025437]
 [-0.2982412  -0.27473122 -0.0545058  ...  0.4884973   1.0955354
   0.18163367]
 [-0.44378173  0.00930707  0.07223696 ...  0.17290184  1.1833246
   0.07897995]]


## Define Model
A fine-tuned model for tweet sentiment analysis will be constructed using the the pre-processing model, the uncased BERT model, one Dense layer, and a Dropout layer.